In [1]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from pathlib import Path


2026-02-06 14:10:06.581579: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-06 14:10:06.591146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-06 14:10:06.605638: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-06 14:10:06.605663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-06 14:10:06.615381: I tensorflow/core/platform/cpu_feature_gua

In [2]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

DATASET_DIR = Path("/home/admin1/PROJETS_KEYCE/cc_transfert/dataset")
VAL_DIR = DATASET_DIR / "val"

assert VAL_DIR.exists()
print("✅ Dataset validation OK")


✅ Dataset validation OK


**Chargement du dataset**

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mn_pre
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_pre
from tensorflow.keras.applications.resnet import preprocess_input as res_pre


In [4]:
def load_val_generator(preprocess_fn):
    gen = ImageDataGenerator(preprocessing_function=preprocess_fn)
    return gen.flow_from_directory(
        VAL_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        shuffle=False
    )


**Chargement des modèles**

In [5]:
MODELS = {
    "MobileNetV2": {
        "path": "mobilenetv2_gc10_finetuned.h5",
        "preprocess": mn_pre
    },
    "EfficientNetB0": {
        "path": "efficientnetb0_gc10_finetuned.h5",
        "preprocess": eff_pre
    },
    "ResNet50": {
        "path": "resnet50_gc10_finetuned.h5",
        "preprocess": res_pre
    }
}

In [6]:
loaded_models = {}

for name, cfg in MODELS.items():
    model = keras.models.load_model(cfg["path"])
    loaded_models[name] = model
    print(f"✅ {name} chargé")


2026-02-06 14:10:08.381674: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-06 14:10:08.424802: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


✅ MobileNetV2 chargé


✅ EfficientNetB0 chargé


✅ ResNet50 chargé


**mesure temps d’inférence**

In [7]:
def measure_inference_time(model, dataset, n_batches=10):
    times = []
    for i, (x, _) in enumerate(dataset):
        if i >= n_batches:
            break
        start = time.time()
        _ = model.predict(x, verbose=0)
        times.append(time.time() - start)

    return (np.mean(times) / x.shape[0]) * 1000  # ms / image


**Benchmark**

In [8]:
results = []

for name, cfg in MODELS.items():
    print(f"\n🔍 Benchmark {name}")

    val_gen = load_val_generator(cfg["preprocess"])
    model = loaded_models[name]

    # Accuracy
    loss, acc = model.evaluate(val_gen, verbose=0)

    # Params
    params = model.count_params()

    # Inference time
    inf_time = measure_inference_time(model, val_gen)

    results.append({
        "Model": name,
        "Val_Accuracy": round(acc, 4),
        "Params_M": round(params / 1e6, 2),
        "Inference_ms": round(inf_time, 2)
    })



🔍 Benchmark MobileNetV2
Found 466 images belonging to 10 classes.

🔍 Benchmark EfficientNetB0
Found 466 images belonging to 10 classes.

🔍 Benchmark ResNet50
Found 466 images belonging to 10 classes.


**Tableau comparatif final**

In [9]:
df_results = pd.DataFrame(results)
df_results = df_results.sort_values("Val_Accuracy", ascending=False)

df_results


,Model,Val_Accuracy,Params_M,Inference_ms
2,ResNet50,0.8648,24.65,25.22
1,EfficientNetB0,0.6116,4.39,20.33
0,MobileNetV2,0.5751,2.59,13.69


**Sauvegarde pour Streamlit**

In [10]:
df_results.to_csv("benchmark_models.csv", index=False)
print("💾 benchmark_models.csv sauvegardé")


💾 benchmark_models.csv sauvegardé
